In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

In [27]:
job_postings_data = pd.read_csv('preprocessed_job_postings.csv')
job_benefits_data = pd.read_csv('benefits.csv')
company_data = pd.read_csv('Companies_stateformat changed.csv')

In [28]:
# DROP Inferred data from job benifits

In [29]:
job_benefits_data = job_benefits_data.drop('inferred', axis=1)

In [30]:
job_benefits_data

,job_id,type
0,3690843087,Medical insurance
1,3690843087,Dental insurance
2,3690843087,401(k)
3,3690843087,Paid maternity leave
4,3690843087,Disability insurance
...,...,...
13756,3701373527,Medical insurance
13757,3701373527,Vision insurance
13758,3701373527,Dental insurance
13759,3701373527,401(k)


In [31]:
company_data

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,At IBM we do more than work. We create. We cre...,7.0,NY,US,Armonk New York,10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,CA,US,Chicago,10017,-,https://www.linkedin.com/company/gehealthcare
2,1021,GE Power,GE Power part of GE Vernova is a world energy ...,7.0,NY,US,Schenectady,12345,1 River Road,https://www.linkedin.com/company/gepower
3,1025,Hewlett Packard Enterprise,Official LinkedIn of Hewlett Packard Enterpris...,7.0,TX,US,Houston,77389,1701 E Mossy Oaks Rd Spring,https://www.linkedin.com/company/hewlett-packa...
4,1028,Oracle,Were a cloud technology company that provides ...,7.0,TX,US,Austin,78741,2300 Oracle Way,https://www.linkedin.com/company/oracle
...,...,...,...,...,...,...,...,...,...,...
6058,3700144594,BYREDO,Beauty can be many things to many people. Our ...,3.0,CA,US,New York,10017,3350 Riverwood Pkwy #1400,https://www.linkedin.com/company/byredo
6059,3700144710,Pros2Plan a division of Spinnaker SCA,Pros2Plan a division of Spinnaker Services LL...,2.0,CO,US,Boulder,80303,3350 Riverwood Pkwy #1400,https://www.linkedin.com/company/pros2plan
6060,3700147810,Ascendo Resources,Ascendo Resources is a certified minority owne...,3.0,FL,US,Coral Gables,33134,2 Alhambra Plaza,https://www.linkedin.com/company/ascendoresources
6061,3700150295,The Crox Group,The Crox Group Head Quartered in Chicago with ...,2.0,IL,US,Lincolnwood,60712,6818 N Lincoln Ave,https://www.linkedin.com/company/the-crox-group


In [32]:
# Drop description , zip code and address from companies 

In [33]:
company_data=company_data.drop('description', axis=1)

In [34]:
company_data=company_data.drop('zip_code', axis=1)

In [35]:
company_data=company_data.drop('address', axis=1)

In [36]:
company_data

,company_id,name,company_size,state,country,city,url
0,1009,IBM,7.0,NY,US,Armonk New York,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,7.0,CA,US,Chicago,https://www.linkedin.com/company/gehealthcare
2,1021,GE Power,7.0,NY,US,Schenectady,https://www.linkedin.com/company/gepower
3,1025,Hewlett Packard Enterprise,7.0,TX,US,Houston,https://www.linkedin.com/company/hewlett-packa...
4,1028,Oracle,7.0,TX,US,Austin,https://www.linkedin.com/company/oracle
...,...,...,...,...,...,...,...
6058,3700144594,BYREDO,3.0,CA,US,New York,https://www.linkedin.com/company/byredo
6059,3700144710,Pros2Plan a division of Spinnaker SCA,2.0,CO,US,Boulder,https://www.linkedin.com/company/pros2plan
6060,3700147810,Ascendo Resources,3.0,FL,US,Coral Gables,https://www.linkedin.com/company/ascendoresources
6061,3700150295,The Crox Group,2.0,IL,US,Lincolnwood,https://www.linkedin.com/company/the-crox-group


In [37]:
job_postings_data

,job_id,company_id,title,max_salary,min_salary,pay_period,formatted_work_type,location,applies,original_listed_time,...,application_url,application_type,expiry,formatted_experience_level,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,133114754,77766802,Sales Manager,82000.0,60000.0,YEARLY,Full-time,"Santa Clarita, CA",6,1.690000e+12,...,unknown,ComplexOnsiteApply,1.700000e+12,Mid-Senior level,1.690000e+12,jobs.smartrecruiters.com,0,FULL_TIME,USD,BASE_SALARY
1,133196985,1089558,Model Risk Auditor,82000.0,60000.0,YEARLY,Contract,"New York, NY",1,1.690000e+12,...,unknown,ComplexOnsiteApply,1.700000e+12,Mid-Senior level,1.690000e+12,jobs.smartrecruiters.com,0,CONTRACT,USD,BASE_SALARY
2,381055942,96654609,Business Manager,82000.0,60000.0,YEARLY,Full-time,"Forney, TX",6,1.690000e+12,...,unknown,ComplexOnsiteApply,1.700000e+12,Mid-Senior level,1.690000e+12,jobs.smartrecruiters.com,0,FULL_TIME,USD,BASE_SALARY
3,529257371,1244539,NY Studio Assistant,82000.0,60000.0,YEARLY,Full-time,"New York, NY",6,1.690000e+12,...,unknown,ComplexOnsiteApply,1.710000e+12,Mid-Senior level,1.690000e+12,jobs.smartrecruiters.com,1,FULL_TIME,USD,BASE_SALARY
4,903408693,3894635,Office Associate,42000.0,37000.0,YEARLY,Full-time,"Albany, GA",5,1.690000e+12,...,unknown,ComplexOnsiteApply,1.710000e+12,Mid-Senior level,1.690000e+12,jobs.smartrecruiters.com,1,FULL_TIME,USD,BASE_SALARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15515,3701373516,74718032,Sanitation Technician,82000.0,60000.0,YEARLY,Part-time,"West Columbia, SC",6,1.690000e+12,...,https://aspirebakeriescareers.com/job/West-Col...,OffsiteApply,1.700000e+12,Entry level,1.690000e+12,aspirebakeriescareers.com,0,PART_TIME,USD,BASE_SALARY
15516,3701373522,38897,Unit Secretary,82000.0,60000.0,YEARLY,Full-time,"Teaneck, NJ",2,1.690000e+12,...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,1.700000e+12,Entry level,1.690000e+12,recruiting.ultipro.com,0,FULL_TIME,USD,BASE_SALARY
15517,3701373523,38897,"Radiology Aide, Perdiem",82000.0,60000.0,YEARLY,Part-time,"Teaneck, NJ",6,1.690000e+12,...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,1.700000e+12,Entry level,1.690000e+12,recruiting.ultipro.com,0,PART_TIME,USD,BASE_SALARY
15518,3701373524,2623,MRI Manager,135000.0,110000.0,YEARLY,Full-time,"New York, NY",6,1.690000e+12,...,https://opportunities.columbia.edu/jobs/mri-ma...,OffsiteApply,1.700000e+12,Mid-Senior level,1.690000e+12,opportunities.columbia.edu,0,FULL_TIME,USD,BASE_SALARY


In [38]:
# Drop original_listed_time,expiry,listed_time,currency,expiry,compensation_type

In [39]:
columns_to_drop= ['original_listed_time', 'expiry', 'listed_time','currency', 'expiry', 'compensation_type']

job_postings_data=job_postings_data.drop(columns_to_drop, axis=1)

In [40]:
job_postings_data

,job_id,company_id,title,max_salary,min_salary,pay_period,formatted_work_type,location,applies,views,job_posting_url,application_url,application_type,formatted_experience_level,posting_domain,sponsored,work_type
0,133114754,77766802,Sales Manager,82000.0,60000.0,YEARLY,Full-time,"Santa Clarita, CA",6,25,https://www.linkedin.com/jobs/view/133114754/?...,unknown,ComplexOnsiteApply,Mid-Senior level,jobs.smartrecruiters.com,0,FULL_TIME
1,133196985,1089558,Model Risk Auditor,82000.0,60000.0,YEARLY,Contract,"New York, NY",1,17,https://www.linkedin.com/jobs/view/133196985/?...,unknown,ComplexOnsiteApply,Mid-Senior level,jobs.smartrecruiters.com,0,CONTRACT
2,381055942,96654609,Business Manager,82000.0,60000.0,YEARLY,Full-time,"Forney, TX",6,25,https://www.linkedin.com/jobs/view/381055942/?...,unknown,ComplexOnsiteApply,Mid-Senior level,jobs.smartrecruiters.com,0,FULL_TIME
3,529257371,1244539,NY Studio Assistant,82000.0,60000.0,YEARLY,Full-time,"New York, NY",6,2,https://www.linkedin.com/jobs/view/529257371/?...,unknown,ComplexOnsiteApply,Mid-Senior level,jobs.smartrecruiters.com,1,FULL_TIME
4,903408693,3894635,Office Associate,42000.0,37000.0,YEARLY,Full-time,"Albany, GA",5,49,https://www.linkedin.com/jobs/view/903408693/?...,unknown,ComplexOnsiteApply,Mid-Senior level,jobs.smartrecruiters.com,1,FULL_TIME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15515,3701373516,74718032,Sanitation Technician,82000.0,60000.0,YEARLY,Part-time,"West Columbia, SC",6,1,https://www.linkedin.com/jobs/view/3701373516/...,https://aspirebakeriescareers.com/job/West-Col...,OffsiteApply,Entry level,aspirebakeriescareers.com,0,PART_TIME
15516,3701373522,38897,Unit Secretary,82000.0,60000.0,YEARLY,Full-time,"Teaneck, NJ",2,7,https://www.linkedin.com/jobs/view/3701373522/...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,Entry level,recruiting.ultipro.com,0,FULL_TIME
15517,3701373523,38897,"Radiology Aide, Perdiem",82000.0,60000.0,YEARLY,Part-time,"Teaneck, NJ",6,3,https://www.linkedin.com/jobs/view/3701373523/...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,Entry level,recruiting.ultipro.com,0,PART_TIME
15518,3701373524,2623,MRI Manager,135000.0,110000.0,YEARLY,Full-time,"New York, NY",6,10,https://www.linkedin.com/jobs/view/3701373524/...,https://opportunities.columbia.edu/jobs/mri-ma...,OffsiteApply,Mid-Senior level,opportunities.columbia.edu,0,FULL_TIME


In [41]:
job_postings_data.to_csv('~/Documents/job_postings_data.csv',index=False, encoding = 'utf-8')


In [42]:
company_data.to_csv('~/Documents/company_data.csv',index=False, encoding = 'utf-8')

In [43]:
job_benefits_data.to_csv('~/Documents/job_benefits_data.csv',index=False, encoding = 'utf-8')